In [1]:
import os
import pandas as pd

extraction_path = '/kaggle/input/soil-classification'

# List the files and directories within the extraction_path
print("Contents of extraction_path:")
print(os.listdir(extraction_path))

# Inspect the contents of the train and test directories
train_path = os.path.join(extraction_path, 'soil_classification-2025', 'train')
test_path = os.path.join(extraction_path, 'soil_classification-2025', 'test')

print("\nContents of train directory:")
print(os.listdir(train_path)[:10]) # Print only first 10 for brevity

print("\nContents of test directory:")
print(os.listdir(test_path)[:10]) # Print only first 10 for brevity

# Read train_labels.csv
train_labels_path = os.path.join(extraction_path, 'soil_classification-2025', 'train_labels.csv')
train_labels_df = pd.read_csv(train_labels_path)
print("\nHead of train_labels.csv:")
display(train_labels_df.head())

# Read test_ids.csv
test_ids_path = os.path.join(extraction_path, 'soil_classification-2025', 'test_ids.csv')
test_ids_df = pd.read_csv(test_ids_path)
print("\nHead of test_ids.csv:")
display(test_ids_df.head())

Contents of extraction_path:
['soil_classification-2025']

Contents of train directory:
['img_3c4ed833.jpeg', 'img_8163dc71.jpg', 'img_e48dfef4.jpg', 'img_83d433d2.jpg', 'img_4b62f891.jpeg', 'img_15c3ff99.jpg', 'img_dacfe3f9.jpg', 'img_febe3434.jpeg', 'img_dbceabbb.jpg', 'img_e9f1d910.jpeg']

Contents of test directory:
['img_0f035b97.jpg', 'img_f13af256.jpg', 'img_15b41dbc.jpg', 'img_cfb4fc7a.jpg', 'img_683111fb.jpg', 'img_c4bd7b3e.jpg', 'img_4ccce0f8.jpg', 'img_86faa98d.jpg', 'img_c448342c.jpg', 'img_e7f7c796.jpg']

Head of train_labels.csv:


,image_id,soil_type
0,img_ed005410.jpg,Alluvial soil
1,img_0c5ecd2a.jpg,Alluvial soil
2,img_ed713bb5.jpg,Alluvial soil
3,img_12c58874.jpg,Alluvial soil
4,img_eff357af.jpg,Alluvial soil



Head of test_ids.csv:


,image_id
0,img_cdf80d6f.jpeg
1,img_c0142a80.jpg
2,img_91168fb0.jpg
3,img_9822190f.jpg
4,img_e5fc436c.jpeg


## Prepare the data for pytorch

### Subtask:
Create PyTorch Datasets and DataLoaders to efficiently load and preprocess the image data and labels for training.


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# Define image transformations
# Training transformations include data augmentation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 as required by ViT
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Test transformations are typically just resizing and normalization
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create a mapping from soil type strings to integers
soil_types = train_labels_df['soil_type'].unique()
soil_type_to_int = {soil_type: i for i, soil_type in enumerate(soil_types)}
int_to_soil_type = {i: soil_type for soil_type, i in soil_type_to_int.items()}

class SoilDataset(Dataset):
    def __init__(self, image_dir, dataframe, transform=None, is_test=False):
        self.image_dir = image_dir
        self.dataframe = dataframe
        self.transform = transform
        self.is_test = is_test
        if not self.is_test:
            self.labels = dataframe['soil_type'].map(soil_type_to_int).tolist()
        self.image_ids = dataframe['image_id'].tolist()


    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB') # Ensure image is in RGB format

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, img_name # Return image and its name for test set
        else:
            label = self.labels[idx]
            return image, label

# Define the image directories
train_image_dir = os.path.join('/kaggle/input/soil-classification', 'soil_classification-2025', 'train')
test_image_dir = os.path.join('/kaggle/input/soil-classification', 'soil_classification-2025', 'test')

# Create Dataset instances
train_dataset = SoilDataset(train_image_dir, train_labels_df, transform=train_transforms)
test_dataset = SoilDataset(test_image_dir, test_ids_df, transform=test_transforms, is_test=True)

# Create DataLoader instances
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")
print(f"Number of batches in train_dataloader: {len(train_dataloader)}")
print(f"Number of batches in test_dataloader: {len(test_dataloader)}")
print(f"Soil type to integer mapping: {soil_type_to_int}")
print(f"Integer to soil type mapping: {int_to_soil_type}")

# Example of getting a batch from train_dataloader
images, labels = next(iter(train_dataloader))
print(f"\nExample batch from train_dataloader:")
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

# Example of getting a batch from test_dataloader
test_images, test_image_names = next(iter(test_dataloader))
print(f"\nExample batch from test_dataloader:")
print(f"Images shape: {test_images.shape}")
print(f"Image names (first 5): {test_image_names[:5]}")

Number of training samples: 1222
Number of test samples: 341
Number of batches in train_dataloader: 39
Number of batches in test_dataloader: 11
Soil type to integer mapping: {'Alluvial soil': 0, 'Clay soil': 1, 'Red soil': 2, 'Black Soil': 3}
Integer to soil type mapping: {0: 'Alluvial soil', 1: 'Clay soil', 2: 'Red soil', 3: 'Black Soil'}

Example batch from train_dataloader:
Images shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])

Example batch from test_dataloader:
Images shape: torch.Size([32, 3, 224, 224])
Image names (first 5): ('img_cdf80d6f.jpeg', 'img_c0142a80.jpg', 'img_91168fb0.jpg', 'img_9822190f.jpg', 'img_e5fc436c.jpeg')


## Define the vit model

### Subtask:
Implement or load a pre-trained Vision Transformer model using PyTorch.


In [ ]:

import torchvision.models as models
from torchvision.models import ViT_B_16_Weights # Import the specific weights class

# 1. Import the vit_base_patch16_224 model and 2. Load a pre-trained version
# Use weights=models.ViT_B_16_Weights.IMAGENET1K_V1 for pretrained weights
model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)

# 3. Modify the classifier layer (fc) to match the number of unique soil classes
num_classes = len(soil_type_to_int) # Get the number of unique classes from the previously created mapping
model.heads.head = torch.nn.Linear(model.heads.head.in_features, num_classes)

# 4. Move the model to the appropriate device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model loaded and moved to device: {device}")
print(f"Number of output classes in the modified classifier: {model.heads.head.out_features}")

## Set up the training environment

### Subtask:
Define the loss function, optimizer, and potentially a learning rate scheduler for training.


In [6]:
import torch.optim as optim
from torch.optim import lr_scheduler

# setting the loss to cross entropy loss for multi label calssification 
criterion = torch.nn.CrossEntropyLoss()
# setting the optimizer to adamW
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
# using step learning rate scheduling 
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


print("Loss Function:", criterion)
print("Optimizer:", optimizer)
print("Learning Rate Scheduler:", scheduler)

Loss Function: CrossEntropyLoss()
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.0001
    lr: 0.0001
    maximize: False
    weight_decay: 0.01
)
Learning Rate Scheduler: <torch.optim.lr_scheduler.StepLR object at 0x7f0ec65916d0>


## Train the model

### Subtask:
Implement the training loop, including forward passes, backward passes, and optimizer steps.


In [8]:
import time


num_epochs = 20
print("Starting training...")
for epoch in range(num_epochs):
    model.train()

    running_loss = 0.0
    start_time = time.time()


    for images, labels in train_dataloader:
        # send everything to gpu 
        images = images.to(device)
        labels = labels.to(device)

        # making the optimizer 0 so that the values dont stack 
        optimizer.zero_grad()

        # forward pass
        outputs = model(images)
    
        # calculating the loss
        loss = criterion(outputs, labels)

        # doing backpropagation 
        loss.backward()

        # doing optimizer step 
        optimizer.step()

        running_loss += loss.item() * images.size(0)


    epoch_loss = running_loss / len(train_dataset)
    epoch_time = time.time() - start_time


    if scheduler is not None:
        scheduler.step()


    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Time: {epoch_time:.2f}s")

print("Training finished.")

Starting training...
Epoch 1/20, Loss: 0.1437, Time: 34.10s
Epoch 2/20, Loss: 0.0961, Time: 34.84s
Epoch 3/20, Loss: 0.0803, Time: 34.11s
Epoch 4/20, Loss: 0.0582, Time: 38.18s
Epoch 5/20, Loss: 0.0451, Time: 39.82s
Epoch 6/20, Loss: 0.0633, Time: 39.89s
Epoch 7/20, Loss: 0.0312, Time: 38.01s
Epoch 11/20, Loss: 0.0134, Time: 35.02s
Epoch 12/20, Loss: 0.0128, Time: 33.51s
Epoch 13/20, Loss: 0.0131, Time: 33.33s
Epoch 14/20, Loss: 0.0105, Time: 33.78s
Epoch 15/20, Loss: 0.0095, Time: 36.84s
Epoch 16/20, Loss: 0.0114, Time: 35.43s
Epoch 17/20, Loss: 0.0090, Time: 36.11s
Epoch 18/20, Loss: 0.0103, Time: 33.46s
Epoch 19/20, Loss: 0.0100, Time: 33.49s
Epoch 20/20, Loss: 0.0095, Time: 33.60s
Training finished.


## Evaluate the model and generate submission file

### Subtask:
Evaluate the trained model on the test dataset and generate a submission file with predictions.

In [9]:

import pandas as pd


# setting the model to evaluation mode
model.eval()


predictions = []
image_ids = []


with torch.no_grad():

    for images, img_names in test_dataloader:
        # testing the model
        images = images.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        predicted_soil_types = [int_to_soil_type[pred.item()] for pred in predicted]

        predictions.extend(predicted_soil_types)
        image_ids.extend(img_names)


submission_df = pd.DataFrame({'image_id': image_ids, 'soil_type': predictions})


print("Submission DataFrame head:")
display(submission_df.head())

# making the submission.csv file for competition 
submission_df.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created.")

Submission DataFrame head:


,image_id,soil_type
0,img_cdf80d6f.jpeg,Alluvial soil
1,img_c0142a80.jpg,Alluvial soil
2,img_91168fb0.jpg,Alluvial soil
3,img_9822190f.jpg,Alluvial soil
4,img_e5fc436c.jpeg,Alluvial soil



Submission file 'submission.csv' created.


## Calculate F1 score

In [11]:

from sklearn.metrics import f1_score



model.eval()

train_predictions = []
train_true_labels = []

with torch.no_grad():
    # calculating the f1 score on train dataset because true labels are not given of test data 
    for images, labels in train_dataloader:

        images = images.to(device)
        labels = labels.to(device)


        outputs = model(images)

        # basically doing argmax here 
        _, predicted = torch.max(outputs.data, 1)


        train_true_labels.extend(labels.cpu().tolist())
        train_predictions.extend(predicted.cpu().tolist())

# calculating the weighted f1 score 
f1_train = f1_score(train_true_labels, train_predictions, average='weighted')

print(f"Calculated F1 Score on Training Data: {f1_train:.4f}")




Calculated F1 Score on Training Data: 0.9975
